In [ ]:

n_paths = 5
alpha = 1
beta = 0.1

chars = "abcdefghijklmnopqrstuvwxyz -"

alphabet = dict(zip(chars, range(28) ))
rev_alphabet = dict(zip(range(28), (chars)))
T = char_probs.shape[0]



def stimes(factors):
    return np.exp(np.sum(np.log(factors)))
   
def sexp(base, exp):
    return np.exp(np.log(base) * exp)
    
def top_keys(d, k, beta):
    arr = np.asarray(d.values())
    string_lens = np.asarray([len(string) for string in d.keys()])
    len_multipliers = string_lens ** beta
    top_k_idxs = argsort(arr * string_len)[-k:] 
    top_k_keys = [d.keys()[i] for i in top_k_idxs]
    return top_k_keys

def beam_search():
    
    Z = [[] for t in range(T)]
    p_s_b = [{} for t in range(T)]
    p_s_nb = [{} for t in range(T)]
    p_s = [{} for t in range(T)]

    # Just an initialization that works...
    p_s_nb[0] = {"":1.}
    p_s_b[0] = {"": 0.}

    p_s[0] = {"":1.}
    Z[0] = [""]
    
    # Iterate over input timesteps
    for t in range(1, T+1):

        # Consider all of the strings which were candidates given data up to time t-1
        # and compute the probabilities of 'some' new strings given data up to time t
        # 'some' = the current strings and 1-character extensions of them
        for string in Z[t-1]:

            # Probs for existing candidates when the final model out is blank, and when it is not blank
            p_s_b[t][string] = stimes( p_s[t-1][string] , char_probs[t, alphabet["-"]])
            p_s_nb[t][string] = 0. if string=="" else stimes(p_s_nb[t-1][string], char_probs[t, alphabet[string[-1]]]) # plus some other stuff???

            # Now consider extensions of one character
            for c in chars[:-1]:
                # When extension is not the same as final character
                if c != string[-1]:
                    p_s_nb[t][string+c] = stimes([char_probs[t, alphabet[c]] , p_s[t-1][string] , sexp(lm_p(c, string), alpha)])
                # When extension is the same as final character
                else:
                    p_s_nb[t][string+c] = stimes([char_probs[t, alphabet[c]], p_s_b[t-1][string] , sexp(lm_p(c, string), alpha)]) 

            # Set the probability of a string given input data up to time t as the sum of the marginals over t'th model out
            p_s[t] = {s: p_s_b[t][s] + p_s_nb[t][s] for s in p_s_b[t].keys()}
            # Take the top k strings 
            Z[t] = top_keys(p_s[t], n_paths, beta)
    return Z[T]